# Advanced Tutorial: Named Entity Recognition using a Bi-LSTM with the Conditional Random Field Algorithm

Tutorial Link:  https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html

A bit more basic is the Bi-Directional Recurrent Neural Network (a more complicated diagram will be shown below to augment, this is the general idea):

<img src="https://raw.githubusercontent.com/PythonWorkshop/intro-to-nlp-with-pytorch/master/images/bilstm_flow.png" width="50%">

It does however show sequences that are embedded used to make predictions.  They are trained on both past as well as future information from the given data.  This will make more sense shortly.

In [1]:
import sys
{sys.executable}

{'/Library/Frameworks/Python.framework/Versions/3.6/bin/python3'}

In [ ]:
# ! {sys.executable} -m pip install jdc bs4 requests

In [3]:
import torch
torch.__version__

'1.0.0.dev20181102'

In [4]:
# Imports for ths tutorial
from bs4 import BeautifulSoup
import requests
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import jdc

torch.manual_seed(1)

## Outline

* Definitions

    * Bi-LSTM
    * CRF and potentials
    * Viterbi

* Helper Functions

* Create the Network

* Train

* Evaluate

* Exercise

## Definitions


### Bi-LSTM (Bidirectional-Long Short-Term Memory)

As we saw, an LSTM addresses the vanishing gradient problem of the generic RNN by adding cell state and more non-linear activation function layers to pass on or attenuate signals to varying degrees.  However, the main limitation of an LSTM is that it can **only account for context from the past**, that is, the hidden state, h_t, takes only past information as input.

#### Named Entity Recognition Task

For the task of **Named Entity Recognition (NER)** it is helpful to have context from past as well as the future, or left and right contexts.  This can be addressed with a Bi-LSTM which is two LSTMs, one processing information in a forward fashion as we have already seen and another LSTM that processes the sequences in a reverse fashion giving the future context.  _That second LSTM is just reading the sentence in reverse._  

The hidden states from both LSTMs are then concatenated into a final output layer or vector.

### Conditional Random Field

We don't have to stop at the output vector from the Bi-LSTM!  We're not at our tag for the entity, yet.  We need to understand costs of moving from one tag to the next (or staying put on a tag, even).

In a CRF, we have the concept of a _transition matrix_ which is the costs associated with transitioning from one tag to another - a transition matrix is calculated/trained for each time step.  It is used in the determination of the best path through all potential sequences.

Say **B** is the tag for the beginning of an entity, **I** signifies that we are inside an entity (will follow a **B**) and **O** means we are outside an entity. 

Next, is an example of B-I-O schema labeling for finding nouns in a sentence (by the way, there are a myriad of other schemas out there - see [Referenes](#references) for some more).

| Word | Schema Tag |
| --- | --- |
| She | B |
| was | O |
| born | O |
| in | O |
| North | B |
| Carolina | I |
| but | O |
| grew | O |
| up | O |
| in | O |
| Texas | B |

Let's look at the transition matrix for the costs of moving from one tag (using our B-I-O schema) to the next (remember our Bi-LSTM is understanding both the forward and reverse ordering to get more accurate boundaries for the named entities).

<img src="../images/crf_transition_matrix.png" width="60%">

The mathematical derivations for calculating this matrix and decoding it is beyond the scope of this workshop, however if you wish to learn more see [this](http://www.cs.columbia.edu/~mcollins/crf.pdf) article.

### Viterbi Algorithm

If each Bi-LSTM instance (time step) has an associated output feature map and CRF transition and emission values, then each of these time step outputs will need to be decoded into a path through potential tags and a final score determined.  This is the purpose of the Viterbi algorithm, here, which is commonly used in conjunction with CRFs.

Specifically, the Viterbi algorithm finds the optimal path through a sequence given a cost function by tracing backwards through a graph of all possible paths.  There are computational tricks to finding this path in the high dimensional space and we will see this in the code below (`_forward_backwards_trick`).

Here, let's see a simple example of just the Viterbi algorithm.  The simplicity of Viterbi is that at each time step, it "looks to the left" to find that best path and then moves to the right, repeating this "look to the left" until a "survivor path" or optimal path is found with the last column being the possible tags.  The score may also be found by tracing backwards along this path and using the metric decided upon.

In this example the optimal score (via a metric) is the lowest one, however, one could also look for the highest scoring path if another metric is used as is shown next.

<img src="https://raw.githubusercontent.com/PythonWorkshop/intro-to-nlp-with-pytorch/master/images/viterbi.png" width="70%">


Getting more realistic...

With regards to our NER work here, below is an example of a "survivor" path within the context of the linear-CRF where we are trying to find the highest scoring path through a sequence (giving us the tags and final score).  The transition matrix values are represented by the arrows and a sequence vector is shown as part of the overall cost function.

<img src="../images/linear_crf_example.png" width="70%">

### Putting it All Together

Here we have word embeddings as the data for the forward and reverse LSTMs.  The resulting forward vector (V_f) and backwards vector (V_b or Output layer, here) are concatenated into a final vector (V_o) that feeds into the CRF layer and is decoded via the Viterbi algorithm (part of CRF layer) into the optimal tag for that input.  Note, the initial values for the Hidden inputs for each LSTM (forward and reverse) are often a vector of random numbers.

<br><br>

<img src="https://raw.githubusercontent.com/PythonWorkshop/intro-to-nlp-with-pytorch/master/images/blstm_crf_details.png" width="70%">


<div  align="right"><a href="https://www.sciencedirect.com/science/article/pii/S1532046417300977">Reference</a></div>

> For a more in-depth discussion, see this excellent post describing the Bi-LSTM, CRF and usage of the Viterbi Algorithm (among other NER concepts and equations): [post](https://guillaumegenthial.github.io/sequence-tagging-with-tensorflow.html).

## Helper Functions

In [5]:
def argmax(vec):
    """Return the argmax as a python int"""
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    """
    Input:
        seq - the sequence (array)
        to_ix - the indices to which seqence values are converted (dict)
        
    Output:
        Numerical tensor
        """
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


def log_sum_exp(vec):
    """Compute log sum exp in a numerically stable way for the forward algorithm.
    
    Note:  torch.expand creates a new dimension and broadcasts the values into it."""
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

## Create the Network

In [6]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        """Initialize network."""
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim # represents both directions
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.  This is how the CRF stores
        # costs.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag in the
        # transition matrix of the CRF
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

In [7]:
%%add_to BiLSTM_CRF

def init_hidden(self):
    """Two tensors to hold hidden states, one for each
    LSTM direction with dimensions of (num_layers, 
    minibatch, hidden_dim)"""
    # Minibatch small because small dataset below
    return (torch.randn(2, MINIBATCH_SIZE, self.hidden_dim // 2),
            torch.randn(2, MINIBATCH_SIZE, self.hidden_dim // 2))


In [8]:
%%add_to BiLSTM_CRF


def _forward_backwards_trick(self, feats):
    """Core magic of the Conditional Random Field.  
    
    Input:
        The word embeddeding vectors for a sentence
    
    Since we’re using PyTorch to compute gradients for us, 
    we technically only need the forward part of the forward-backward 
    algorithm """
    # Do the forward algorithm to compute the partition function
    # The "full" function fills in with a default value
    # Here we create a 1x3 matrix
    init_alphas = torch.full((1, self.tagset_size), -10000.)
    
    # init_alphas holds costs of moving from the START_TAG
    # to B, I or O
    # Let's set the cost of staying at the start tag
    init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

    forward_var = init_alphas

    # Iterate through the sentence
    for feat in feats:
        alphas_t = []  # The forward tensors at this timestep
        for next_tag in range(self.tagset_size):
            # broadcast (copy into new dim) the emission score: 
            # it is the same regardless of
            # the previous tag
            emit_score = feat[next_tag].view(
                1, -1).expand(1, self.tagset_size)
            # the ith entry of trans_score is the score of transitioning to
            # next_tag from i
            trans_score = self.transitions[next_tag].view(1, -1)
            # The ith entry of next_tag_var is the value for the
            # edge (i -> next_tag) before we do log-sum-exp
            next_tag_var = forward_var + trans_score + emit_score
            # The forward variable for this tag is log-sum-exp of all the
            # scores.
            alphas_t.append(log_sum_exp(next_tag_var).view(1))
        # Concatenate the alphas_t (cost at timesteps) for feature
        forward_var = torch.cat(alphas_t).view(1, -1)
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    alpha = log_sum_exp(terminal_var)
    return alpha

In [9]:
%%add_to BiLSTM_CRF

def _get_lstm_features(self, sentence):
    """Compute output vector of BLSTM - used in 
    the forward pass of network"""
    self.hidden = self.init_hidden()
    embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
    lstm_out, self.hidden = self.lstm(embeds, self.hidden)
    lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
    lstm_feats = self.hidden2tag(lstm_out)
    return lstm_feats

In [10]:
%%add_to BiLSTM_CRF

def _score_sentence(self, feats, tags):
    """Gives the score of a provided tag sequence"""
    score = torch.zeros(1)
    tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
    for i, feat in enumerate(feats):
        score = score + \
            self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
    score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
    return score

In [11]:
%%add_to BiLSTM_CRF

def _viterbi_decode(self, feats):
    """Implements Viterbi algorithm for finding most likely sequence of labels.
    Used in the forward pass of the network.

    We take the maximum over the previous states as opposed to the sum. 
    Input:
        loglikelihoods: torch tensor.
    Output:
        tuple. The first entry is the loglikelihood of this sequence. The second is 
        the most likely sequence of labels. 
    """
    backpointers = []

    # Initialize the viterbi variables in log space
    init_vvars = torch.full((1, self.tagset_size), -10000.)
    init_vvars[0][self.tag_to_ix[START_TAG]] = 0

    # forward_var at step i holds the viterbi variables for step i-1
    forward_var = init_vvars
    for feat in feats:
        bptrs_t = []  # holds the backpointers for this step
        viterbivars_t = []  # holds the viterbi variables for this step

        for next_tag in range(self.tagset_size):
            # next_tag_var[i] holds the viterbi variable for tag i at the
            # previous step, plus the score of transitioning
            # from tag i to next_tag.
            # We don't include the emission scores here because the max
            # does not depend on them (we add them in below)
            next_tag_var = forward_var + self.transitions[next_tag]
            best_tag_id = argmax(next_tag_var)
            bptrs_t.append(best_tag_id)
            viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
        # Now add in the emission scores, and assign forward_var to the set
        # of viterbi variables we just computed
        forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
        backpointers.append(bptrs_t)

    # Transition to STOP_TAG
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    best_tag_id = argmax(terminal_var)
    path_score = terminal_var[0][best_tag_id]

    # Follow the back pointers to decode the best path.
    best_path = [best_tag_id]
    for bptrs_t in reversed(backpointers):
        best_tag_id = bptrs_t[best_tag_id]
        best_path.append(best_tag_id)
    # Pop off the start tag (we don't want to return that to the caller)
    start = best_path.pop()
    assert start == self.tag_to_ix[START_TAG]  # Sanity check
    best_path.reverse()
    return path_score, best_path

In [12]:
%%add_to BiLSTM_CRF

def neg_log_likelihood(self, sentence, tags):
    """Calculate the negative log likelihood given a sequence and labels"""
    feats = self._get_lstm_features(sentence)
    forward_score = self._forward_backwards_trick(feats)
    gold_score = self._score_sentence(feats, tags)
    return forward_score - gold_score

In [13]:
%%add_to BiLSTM_CRF

def forward(self, sentence):
    """The forward pass function for training the network"""
    # Get the emission scores (output layer) from the 
    # BiLSTM 
    lstm_feats = self._get_lstm_features(sentence)

    # Find the best path, given the emission scores from BiLSTM
    score, tag_seq = self._viterbi_decode(lstm_feats)
    return score, tag_seq

In [14]:
text = "'place the chicken celery carrots onions parsnip (if using) parsley peppercorns bay leaves and salt in a large soup pot and cover with cold water by 1 inch bring to a boil over high heat then immediately reduce the heat to very low adjust the heat until the soup is smiling: barely moving on the surface with an occasional bubble breaking through cook uncovered until the chicken is very tender and falling off the bone 1 to 1 1/2 hours when cool enough to handle use tongs to transfer chicken from the pot to a container taste the broth and continue to simmer it until it is concentrated and tasty strain broth through a fine sieve (or a colander lined with cheesecloth) into a separate container discard all the solids from the strainer (or reserve the vegetables chill and serve with vinaigrette if you wish) refrigerate chicken pieces and broth separately for at least 8 hours (or up to 3 days) until a thick layer of yellow fat has risen to the top of the broth when ready to finish the soup use your fingers to separate chicken breast meat from bones and skin discard bones and skin use two forks to pull the breast meat apart into soft chunks or use a knife and cut into bite-size pieces (reserve dark meat for another use) skim chicken fat from top of broth and set aside place 3 tablespoons of the fat in a soup pot with a lid add leeks stir to coat and heat over medium heat until leeks begin to fry then reduce the heat to a gentle sizzle and cook stirring often until slightly softened about 3 minutes add carrots sprinkle with salt stir and cover the pot cook until vegetables are just tender about 5 minutes more (keep in mind that vegetables will continue to cook in the soup) do not brown pour broth into pot with vegetables and heat to a simmer add noodles and simmer until heated through soft and plumped with chicken broth add the breast meat then taste broth and add salt and pepper to taste for best flavor soup should have some golden droplets of fat on top; if needed add more chicken fat one teaspoon at a time serve immediately in a tureen or from the pot sprinkling each serving with herbs'"

In [15]:
def remove_punct(text):
    """Remove punctuation from a piece of text"""
    punct = list(".,()-")
    for p in punct:
        text = text.replace(p, '')
    return text
    
text = remove_punct(text)

In [16]:
# Create a lookup dict for all possible words and record their index
word_to_ix = {}
for word in text.split():
    if word not in word_to_ix:
        word_to_ix[word] = len(word_to_ix)

## Train

Problem statement:  we wish to find the equipment and ingredients in recipes so we can buy what we need to make our dinner.

In [17]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4
MINIBATCH_SIZE = 1

In [18]:
# Make up some training data - this is from a NYTimes recipe
# The ingrediants and equipment are labeled as entities here.
training_data = [(
    "strain broth through a fine sieve or a colander lined with cheesecloth into a separate container".split(),
    "O B O O B I O O B O O B O O O O".split()
), (
    "pour broth into pot with vegetables and heat to a simmer".split(),
    "O B O B O B O O O O O".split()
), (
    "use two forks to pull the breast meat apart into soft chunks or use a knife and cut into bitesize pieces".split(),
    "O B I O O O B I O O B I O O O B O O O B I I".split())]


In [19]:
tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

In [20]:
# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print(model(precheck_sent))

(tensor(26.5999), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [21]:
# Make sure prepare_sequence from earlier in the LSTM section is loaded
# again, normally you would do more than 300 epochs, but we have
# toy data
for epoch in range(300):  
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. A lot happens.  Run our forward pass to get features from BLSTM,
        # run the CRF and get the negative log likelihoods and find the best 
        # "path" through sentence with the tags using the viterbi algorithm 
        # (also part of forward pass).
        # BTW our dynamic computational graph is created with the forward pass
        # Returns the forward score - ground truth score (our loss measure)
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients (backprop), and update the 
        # parameters by calling optimizer.step() - optimizer here is 
        # SGD for our CRF
        loss.backward()
        optimizer.step()

In [22]:
ix_to_tag = {0: "B", 1: "I", 2: "O"}

# Sanity check for predictions after training
# No need to accumulate gradients because this is a validation
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    pred =  model(precheck_sent)[1]
    print('Prediction:   ', [ix_to_tag[idx] for idx in pred])
    print('Ground truth: ', training_data[0][1])
    print(training_data[0][0])
# We got it!

Prediction:    ['O', 'B', 'O', 'O', 'B', 'I', 'O', 'O', 'B', 'O', 'O', 'B', 'O', 'O', 'O', 'O']
Ground truth:  ['O', 'B', 'O', 'O', 'B', 'I', 'O', 'O', 'B', 'O', 'O', 'B', 'O', 'O', 'O', 'O']
['strain', 'broth', 'through', 'a', 'fine', 'sieve', 'or', 'a', 'colander', 'lined', 'with', 'cheesecloth', 'into', 'a', 'separate', 'container']


## Evaluate

Let's test our model on an unseen sentence.



In [23]:
# Give some test data
test_data = [(
    "add the breast meat then taste broth and add salt and pepper to taste".split(),
    "O O B I O O B O O B O B O O".split()
)]

In [24]:
with torch.no_grad():
    precheck_sent = prepare_sequence(test_data[0][0], word_to_ix)
    pred =  model(precheck_sent)[1]
    print('Prediction:   ', [ix_to_tag[idx] for idx in pred])
    print('Ground truth: ', test_data[0][1])
    print('Text: ', test_data[0][0])

Prediction:    ['O', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'B', 'O', 'O', 'B', 'O', 'O']
Ground truth:  ['O', 'O', 'B', 'I', 'O', 'O', 'B', 'O', 'O', 'B', 'O', 'B', 'O', 'O']
Text:  ['add', 'the', 'breast', 'meat', 'then', 'taste', 'broth', 'and', 'add', 'salt', 'and', 'pepper', 'to', 'taste']


If the result wasn't very good, try more epochs to create a new trained model.

## Exercise

Use BeautifulSoup Python library and the Requests library to download a recipe of your choosing from https://cooking.nytimes.com/ and find it's steps section.


To get started, you can:

```python
page = requests.get('')
soup = BeautifulSoup(page.content, 'html.parser')
steps = soup.findAll("ol", {"class": "recipe-steps"})
```

Instructions:
1. Clean up the html tags and any strange characters.
2. Label entities with YEDDA:  https://github.com/michhar/YEDDA.
3. Save the labels as text and write code to identify entities as labels for the Bi-LSTM above.
4. Run a Bi-LSTM on this data and validate the predictions against the original labels.
5. Run some test data through your model.

## References

1. [Understanding Bidirectional RNN in PyTorch](https://towardsdatascience.com/understanding-bidirectional-rnn-in-pytorch-5bd25a5dd66)
2. [Conditional Random Field Tutorial in PyTorch](https://towardsdatascience.com/conditional-random-field-tutorial-in-pytorch-ca0d04499463)
3. [Character-level neural network for biomedical named entity recognition](https://www.sciencedirect.com/science/article/pii/S1532046417300977)
4.  [Other named entity tag schemas](https://lingpipe-blog.com/2009/10/14/coding-chunkers-as-taggers-io-bio-bmewo-and-bmewo/)

In [25]:

############### For toggle button ##################

from IPython.display import HTML
from IPython.display import display

# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the hint."></form>''')
display(tag)

############### Write code below ##################

# HINT!  (cleaning text data)
import requests
from bs4 import BeautifulSoup

page = requests.get('https://cooking.nytimes.com/recipes/1018442-chicken-soup-from-scratch')
soup = BeautifulSoup(page.content, 'html.parser')
steps = soup.findAll("ol", {"class": "recipe-steps"})

import re

def cleanhtml(raw_html):
    """Function to clean up the html tags in data."""
    cleanr = re.compile('<.*?>')
    # Remove html tags
    cleantext = re.sub(cleanr, '', raw_html)
    cleantext = cleantext.replace('\n', ' ').rstrip().strip()
    # Remove special quotes
    cleantext = cleantext.replace('“', '').replace('”', '')
    cleantext = cleantext.lower()
    return cleantext

cleansteps = cleanhtml(str(steps[0]))
cleansteps

with open('sample_data.txt', 'w') as f:
    f.write(cleansteps)
    
# This data is for updating training data or to use in the exercise
with open('sample_data.txt', 'r') as f:
    data = f.read()
# print(data)